In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:

# Dict API
result = rs.glm_dict(
    response='ClaimCount',
    terms={'DrivAge': {'type': 'ns'}, 'Region': {'type': 'target_encoding'}},
    data=data,
    family='negbinomial'
).fit()

# Access smooth term results
if result.has_smooth_terms():
    print(f"Total EDF: {result.total_edf:.2f}")
    print(f"GCV: {result.gcv:.4f}")
    for st in result.smooth_terms:
        print(f"  {st.variable}: EDF={st.edf:.2f}, lambda={st.lambda_:.2f}")

Total EDF: 9.83
GCV: 0.2734
  DrivAge: EDF=7.83, lambda=17.76


In [3]:
print(result.summary())

                                 GLM Results                                  

Family:              negbinomial     No. Observations:        678012
Link Function:       (default)       Df Residuals:            678001
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   5

Log-Likelihood:          -92676.8966 Deviance:                185353.7933
AIC:                     185375.7933 Null Deviance:            37380.9687
BIC:                     185501.4894 Pearson chi2:              714103.58
Converged:           True           

------------------------------------------------------------------------------
Variable               Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept           -3.1067     0.0606  -51.273  <0.0001   [ -3.2254,  -2.9879]  ***
ns(DrivAge, 1/9)    -0.1028     0.0083  -12.318  <0.000

In [4]:
result.predict(holdout_data)

array([0.04395678, 0.05124573, 0.08705769, ..., 0.06700595, 0.05215035,
       0.0609028 ], shape=(135957,))

In [3]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [4]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   7

Log-Likelihood:          -84170.0145 Deviance:                126792.3793
AIC:                     168362.0290 Null Deviance:           132328.3090
BIC:                     168482.0961 Pearson chi2:              935544.55
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -3.4955     0.0707  -49.452  <0.0001   [ -3.6341,  -3.3570]  ***
BonusMalus             0.0224     0.0004   56.269  

In [ ]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [22]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21240.95, loss=0.311851, deviance=126748.59, log_likelihood=-84148.12, aic=168318.24, gini=0.2575, auc=0.6288, ae_ratio=1.0079, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1901.19, actual=0.400276, predicted=0.077359, ae_ratio=5.1742), DecileMetrics(decile=2, n=40643, exposure=4171.28, actual=0.244529, predicted=0.090697, ae_ratio=2.6961), DecileMetrics(decile=3, n=40643, exposure=8080.85, actual=0.165082, predicted=0.087264, ae_ratio=1.8918), DecileMetrics(decile=4, n=40643, exposure=13706.8, actual=0.128914, predicted=0.082754, ae_ratio=1.5578), DecileMetrics(decile

In [6]:
predictions = result.predict(holdout_data)

In [7]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [8]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [9]:
predictions2 = loaded.predict(holdout_data)

In [10]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [ ]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

In [ ]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [ ]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

In [2]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [2]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

{'model_summary': {'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=4) + ns(DrivAge, df=4) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 22, 'df_resid': 406417, 'converged': True, 'iterations': 7, 'regularization': {'type': 'ridge', 'alpha': 0.4, 'l1_ratio': 0.0}}, 'train_test': {'train': {'dataset': 'train', 'n_obs': 406439, 'total_exposure': 214930.9, 'total_actual': 21408.0, 'total_predicted': 21530.19, 'loss': 0.310655, 'deviance': 126262.46, 'log_likelihood': -83905.05, 'aic': 167854.11, 'gini': 0.2747, 'auc': 0.6374, 'ae_ratio': 0.9943, 'ae_by_decile': [{'decile': 1, 'n': 40643, 'exposure': 1909.44, 'actual': 0.392262, 'predicted': 0.077322, 'ae_ratio': 5.0731}, {'decile': 2, 'n': 40643, 'exposure': 4144.57, 'actual': 0.234765, 'predicted': 0.091204, 'ae_ratio': 2.5741}, {'decile': 3, 'n': 40643, 'exposure': 8115.13, 'actual': 0.167711, 'predicted': 0.087742, 'ae_ratio': 1.9114}, {'decile': 4, 'n': 40643, 'exposure': 13627.33, 'actual': 0.125263